# GOATHerd Base

In [13]:
import sys

sys.path.append("/xpal-src/goatherd")

from goatherd import *
# Connect to the GOATs
neo4jconfig=get_mgc("neo4j")
print(neo4jconfig)
neo4jdb=Neo4jGoat(neo4jconfig)

{'username': 'neo4j', 'password': 'theansweris42', 'dbname': 'neo4j', 'dburl': 'localhost:7687', 'dbport': '7687'}


In [21]:
labels=[
    'Person',
    'Brand',
    'Company',
    'NGO',
    'Govt Body',
    'Node',
]
def update_keystones(goat):
    k1=goat.add_node(nodeid="__keystone1")
    k2=goat.add_node(nodeid="__keystone2")
    goat.update_labels("__keystone1",labels)
    goat.update_labels("__keystone2",labels)

    k1.isthesameas.add(k2)
    k1.linkedto.add(k2)
    k2.isthesameas.add(k1)
    k2.linkedto.add(k1)
    goat.repo.save(k2)
    goat.repo.save(k1)

In [22]:
print(neo4jdb.get_compostion())
update_keystones(neo4jdb)
print(neo4jdb.get_compostion())

[{'Node': 1}, {'Organization': 1}, {'Company': 1}]
__keystone1
Adding new node
__keystone2
Adding new node
[{'Person': 2}, {'Govt Body': 2}, {'NGO': 2}, {'Brand': 2}, {'Company': 3}, {'Node': 3}, {'Organization': 1}]


In [11]:
company={"nodeid": "U85300DL2022PTC402649", "companyname": "HCAH SENIOR CARE PRIVATE LIMITED", "urls": ["https://www.tofler.in/hcah-senior-care-private-limited/company/U85300DL2022PTC402649", "https://www.hcah.in"], "labels": ["Company", "Organization", "Node"]}

In [12]:
company


{'nodeid': 'U85300DL2022PTC402649',
 'companyname': 'HCAH SENIOR CARE PRIVATE LIMITED',
 'urls': ['https://www.tofler.in/hcah-senior-care-private-limited/company/U85300DL2022PTC402649',
  'https://www.hcah.in'],
 'labels': ['Company', 'Organization', 'Node']}

In [18]:
n=neo4jdb.add_node(**company)

U85300DL2022PTC402649
Node exists


In [20]:
neo4jdb.update_labels(n.nodeid,company['labels'])

:Company:Node:Organization